# First Order Logic

# Libraries

In [2]:
import pygame
import numpy as np
import copy
from pyswip.prolog import Prolog

pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Environment Theme
#### Don't Change this part

In [6]:
GRID_SIZE = 8
TILE_SIZE = 100
background = (106, 70, 205)

# Environment
#### Don't Change this part

In [7]:
class PygameInit:

    @classmethod
    def initialization(cls):
        grid_size_x = GRID_SIZE
        grid_size_y = GRID_SIZE
        tile_size = TILE_SIZE

        pygame.init()
        screen = pygame.display.set_mode((grid_size_x * tile_size, grid_size_y * tile_size))
        pygame.display.set_caption("FOL")
        clock = pygame.time.Clock()

        return screen, clock

class FirstOrderAngry:
    def __init__(self, template: str):
        self.__grid_size = GRID_SIZE
        self.__tile_size = TILE_SIZE
        self.__template_name = template

        self.__base_grid = self.__generate_grid()
        self.grid = copy.deepcopy(self.__base_grid)
        self.__base_grid = copy.deepcopy(self.grid)

        self.__bird_image = pygame.image.load('Env/icons/angry-birds.png')
        self.__bird_image = pygame.transform.scale(self.__bird_image, (self.__tile_size, self.__tile_size))
        self.__bird_with_background = pygame.Surface((self.__tile_size, self.__tile_size))
        self.__bird_with_background.fill(background)
        self.__bird_with_background.blit(self.__bird_image, (0, 0))

        self.__pig_image = pygame.image.load('Env/icons/pigs.png')
        self.__pig_image = pygame.transform.scale(self.__pig_image, (self.__tile_size, self.__tile_size))
        self.__pig_with_background = pygame.Surface((self.__tile_size, self.__tile_size))
        self.__pig_with_background.fill(background)
        self.__pig_with_background.blit(self.__pig_image, (0, 0))

        self.__rock_image = pygame.image.load('Env/icons/rocks.png')
        self.__rock_image = pygame.transform.scale(self.__rock_image, (self.__tile_size, self.__tile_size))
        self.__rock_with_background = pygame.Surface((self.__tile_size, self.__tile_size))
        self.__rock_with_background.fill(background)
        self.__rock_with_background.blit(self.__rock_image, (0, 0))


    def __generate_grid(self):
        grid = [['T' for _ in range(self.__grid_size)] for _ in range(self.__grid_size)]

        with open(f'Env/templates/{self.__template_name}.txt') as file:
            template_str = file.readlines()

        for i in range(self.__grid_size):
            for j in range(self.__grid_size):
                grid[i][j] = template_str[i][j]

        return grid

    def reset(self):
        self.grid = copy.deepcopy(self.__base_grid)

    def bird_step(self, action):
        bird_pos = self.get_bird_position()
        actions = {
            0: (-1, 0),  # Up
            1: (1, 0),   # Down
            2: (0, -1),  # Left
            3: (0, 1)    # Right
        }

        dx, dy = actions[action]
        new_row, new_col = bird_pos[0] + dx, bird_pos[1] + dy

        if self.__is_valid_for_bird_position(self.grid, new_row, new_col):
            self.grid[bird_pos[0]][bird_pos[1]] = 'T'
            bird_pos = (new_row, new_col)
            self.grid[bird_pos[0]][bird_pos[1]] = 'B'

        is_win = self.is_win()
        return bird_pos, is_win


    def get_bird_position(self):
        for r in range(len(self.grid)):
            for c in range(len(self.grid)):
                if self.grid[r][c] == 'B':
                    return tuple([r, c])


    def is_win(self):
        for r in range(len(self.grid)):
            for c in range(len(self.grid)):
                if self.grid[r][c] == 'P':
                    return False
        return True


    def render(self, screen):
        for r in range(self.__grid_size):
            for c in range(self.__grid_size):
                color = background
                pygame.draw.rect(screen, color, (c * self.__tile_size, r * self.__tile_size, self.__tile_size,
                                                 self.__tile_size))
                if self.grid[r][c] == 'E':
                    screen.blit(self.__egg_with_background, (c * self.__tile_size, r * self.__tile_size))

                if self.grid[r][c] == 'B':
                    screen.blit(self.__bird_with_background, (c * self.__tile_size, r * self.__tile_size))

                if self.grid[r][c] == 'R':
                    screen.blit(self.__rock_with_background, (c * self.__tile_size, r * self.__tile_size))

                if self.grid[r][c] == 'P':
                    screen.blit(self.__pig_with_background, (c * self.__tile_size, r * self.__tile_size))

        for r in range(self.__grid_size + 1):
            pygame.draw.line(screen, (0, 0, 0), (0, r * self.__tile_size), (self.__grid_size * self.__tile_size,
                                                                            r * self.__tile_size), 2)
        for c in range(self.__grid_size + 1):
            pygame.draw.line(screen, (0, 0, 0), (c * self.__tile_size, 0), (c * self.__tile_size,
                                                                            self.__grid_size * self.__tile_size), 2)

    @classmethod
    def __is_valid_for_bird_position(cls, grid, new_row, new_col):
        return (0 <= new_row < len(grid)
                and 0 <= new_col < len(grid)
                and grid[new_row][new_col] != 'R')

In [8]:
env = FirstOrderAngry(template='simple')

with open("grid.pl", "a") as file:
    grid = env.grid

    grid_size = len(grid)

    # Write grid size fact
    file.write("% Define the grid size\n")
    file.write(f"grid_size({grid_size}, {grid_size}).\n\n")

    # Initialize fact lists
    bird_positions = []
    pig_positions = []
    rock_positions = []
    tile_positions = []

    # Iterate over the grid to categorize facts
    for row in range(grid_size):
        for col in range(grid_size):
            cell = grid[row][col]
            if cell == "B":
                bird_positions.append(f"bird({row}, {col}).\n")
            elif cell == "P":
                pig_positions.append(f"pig({row}, {col}).\n")
                tile_positions.append(f"tile({row}, {col}).\n")
            elif cell == "R":
                rock_positions.append(f"rock({row}, {col}).\n")
            elif cell == "T":
                tile_positions.append(f"tile({row}, {col}).\n")

    # Write bird facts
    file.write("% Bird position\n")
    file.writelines(bird_positions)
    file.write("\n")

    # Write pig facts
    file.write("% Pig positions\n")
    file.writelines(pig_positions)
    file.write("\n")

    # Write rock facts
    file.write("% Rock positions\n")
    file.writelines(rock_positions)
    file.write("\n")

    # Write tile facts
    file.write("% Tile positions (valid positions to move)\n")
    file.writelines(tile_positions)
    file.write("\n")


print("Grid exported to 'grid.pl'")



Grid exported to 'grid.pl'


In [12]:
env.grid

[['B', 'T', 'T'], ['T', 'R', 'R'], ['P', 'T', 'T']]

# Knowledge base

In [4]:
prolog = Prolog()
prolog.consult("x.pl")

PrologError: Caused by: 'consult('x.pl')'. Returned: 'error(existence_error(source_sink, x.pl), _1158)'.

# Game

In [25]:
env = FirstOrderAngry(template='simple')

screen, clock = PygameInit.initialization()
FPS = 8

env.reset()

running = True
while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()

    # select actions based on the extracted path
    action = np.random.choice([0, 1, 2, 3])
    bird_pos, is_win = env.bird_step(action)
    env.render(screen)
    if  is_win:
        print(f'Win')
        running = False

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()

Win
